# TTN遺伝子のイントロン同士の配列相同性をBLASTで評価する

**目的**
イントロン同士の配列を比較して、相同性を確認する<br>
相同性とスプライシングパターンの関連がないかを検証

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
import matplotlib.pyplot as plt
import pandas as pd

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

import subprocess
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 準備

In [ ]:
# ディレクトリのリセット
cmd1 = "rm -rf ../data/blast"
cmd2 = "mkdir -p ../data/blast/db/ ../data/blast/fasta/ ../data/blast/result/"
subprocess.call(cmd1.split())
subprocess.call(cmd2.split())

In [ ]:
# クラスのインスタンス化
human = Seq_count()

# gbkファイルの読み込み
human.read_gbk('../data/gbk/human_ttn.gb')

# 遺伝子全長配列
seq = human.gDNA_seq()

## BLASTの設定

In [ ]:
# すべてのイントロン配列をfasta形式で保存
with open("../data/blast/fasta/ttn_intron.fasta","w") as f:
    for n,i in enumerate(human.intron_list()):
        f.write(f">intron{str(n+1).zfill(3)}\n")
        f.write("".join(seq[i[0]:i[1]]))
        f.write("\n")

# すべてのイントロン配列を個別にfasta形式で保存
for n,i in enumerate(human.intron_list()):
    with open(f"../data/blast/fasta/ttn_intron{str(n+1).zfill(3)}.fasta","w") as f:
        f.write(f">intron{str(n+1).zfill(3)}\n")
        f.write("".join(seq[i[0]:i[1]]))

In [ ]:
# BLASTデータベースの作成
cmd = "makeblastdb -in ../data/blast/fasta/ttn_intron.fasta -dbtype nucl -parse_seqids -out ../data/blast/db/ttn_intron"
subprocess.run(cmd.split())

## BLASTの実行

In [ ]:
# すべてのイントロンに対してBLASTを実行
for i in range(human.intron_num()):
    fmt = 6
    cmd = f"blastn -query ../data/blast/fasta/ttn_intron{str(i+1).zfill(3)}.fasta -db ../data/blast/db/ttn_intron -out ../data/blast/result/ttn_intron{str(i+1).zfill(3)}.txt -outfmt {fmt}"
    subprocess.run(cmd.split())

## BLASTの結果の整理

In [ ]:
# BLAST結果のファイルリスト
ls = glob.glob("../data/blast/result/ttn_intron*.txt")
ls.sort(key = lambda x: int(x.split("/")[-1].split(".")[0].split("n")[-1]))

In [ ]:
# BLAST結果ファイルの読み込みと結合
df = pd.read_csv(ls[0], sep='\t', header=None)
for i in ls[1:]:
    tmp = pd.read_csv(i, sep='\t', header=None)
    df = pd.concat([df, tmp])
df.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
df

In [ ]:
df.to_csv('../data/blast/blast.csv')